# Visualisation de données géo-spatiales

On se propose ici d'analyser le déplacement de 182 utilisateurs équipés de balises GPS. Ce projet est nommé Geolife et est porté par Microsoft.

Le TP consiste à vous familiariser avec les données géo-spatiales et la librairie Kepler.

## I. Import des données
Afin de travailler sur un jeu de données propre pour l'analyse, certaines étapes de préprocessing sont nécessaires.

a) Importer les données dans un DataFrame `df`

Nous utiliserons un extract fourni : **geolife_data_500k_sample.csv**

In [ ]:
import pandas as pd

df = pd.read_csv('/workspace/data/geolife_data_500k_sample.csv')
df.head()

b) Exécutez le code suivant pour renommer les colonnes, changer le format de la colonne Date_Time et enfin ne garder que les positions latitude / longitude qui sont non nulles. Ces étapes sont classiques lors de la manipulation de ce genre de données.

In [ ]:
# renommer les colonnes
df.rename({'Id_user': 'id', 'Date_Time': 'datetime', 'Id_perc': 'id_traj'}, axis=1, inplace=True)

# conversion de la date en string en objet datetime64
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'
df['datetime'] = pd.to_datetime(df['datetime'], format=DATETIME_FORMAT)

# Supression des lignes sans géolocalisation
df = df[(~df['Latitude'].isnull()) & (~df['Longitude'].isnull())]

c) Pour le TD, on ne va garder que les trajectoires des utilisateurs entre 1er Septembre 2009 et le 1er Octobre 2009 exclus

In [ ]:
print(df.shape)
df = df[(df['datetime'] >= '2009-09-01 00:00:00') & (df['datetime'] < '2009-10-01 00:00:00')]
print(df.shape)

d) Afficher le nombre de points par id

In [ ]:
# conversion de l'id en string (utile pour que kepler l'interprete en string)
df['id'] = df['id'].apply(lambda x: 'user_' + str(x))

In [ ]:
df.id.value_counts()

# II. Visualisation basique avec Kepler
Cette partie a pour but de prendre en main les fonctionnements de bases de l'outil de visualisation Kepler afin d'analyser les données de ces trajectoires.

In [ ]:
import json
from keplergl import KeplerGl

a) Affichez les données brutes sur Kepler

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.output_result { max-width:100% !important; }</style>")) 

In [ ]:
mymap = KeplerGl(data={'df': df})
#======= special correction ========
#with open('/workspace/data/ma_config_correction.json', "r") as f:
#    conf = json.load(f)
#mymap.config = conf
#===================================
mymap

b) Attribuez une couleur par ID depuis l'interface de Kepler et afficher la legende

c) Utiliser les filtres pour filtrer sur le temps et afficher la timeline

d) Utilisez les filtres pour filtrer sur l'ID = user_85

e) Masquez le layer 'Point' et ajouter un layer de type 'Heatmap'

f) Affichez la configuration actuelle de Kepler

In [ ]:
mymap.config

g) Enregistrez votre configuration de Kepler dans un fichier nommé 'ma_config.json'

In [ ]:
with open('/workspace/data/ma_config.json', "w") as f:
    f.write(json.dumps(mymap.config, indent=4))

h) Rechargez le fichier de config 'ma_config.json' puis réafficher Kepler (via KeplerGL) en utilisant cette configuration.

Vous devriez alors avoir un Kepler avec uniquement une heatmap du user_85.

In [ ]:
with open('/workspace/data/ma_config.json', "r") as f:
    conf = json.load(f)

mymap = KeplerGl(height=600, data={'df': df})
mymap.config = conf
mymap

# III. Visualisation avancée avec Kepler
Cette partie permet de prendre en main des structures de données plus poussées telles que les trajectoires.

In [ ]:
from datetime import timezone

a) Créez à partir des points, un seul dataframe contenant tous les segments de trajet pour chaque ID.


L'objectif est de créer une ligne par segment de déplacement. Il y a donc une latitude/longitude pour le point de départ et une autre pour le point d'arrivé. Le dataset final doit etre constitué de tous ces segments pour constituer l'ensemble des déplacements de l'individus, réalisés dans l'ordre chronologique.

Complétez le code ci-dessous pour y parvenir.

Indices : vous pouvez utiliser la fonction [shift](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html) sur la variables `rows` des éléments groupés pour reconstruire un dataframe avec un point source et un point destination sur une même ligne avec les dates respectives.
Ce que je veux : une ligne = un segment de trajectoire avec un point source et un point destination

Exemple :
```python
df_points = pd.DataFrame([['lat_1', 'lon_1'],
                          ['lat_2', 'lon_2'],
                          ['lat_3', 'lon_3']],
                         columns=['Latitude', 'Longitude'])

# résultat
source_lat, source_lon, destination_lat, destination_lon
lat_1,      lon_1,      lat_2,           lon_2
lat_2,      lon_2,      lat_3,           lon_3
```

In [ ]:
df.head()

In [ ]:
%%time

segments = []
for idx, rows in df.sort_values('datetime', ascending=True).groupby(['id', 'id_traj']):
    id_user = idx[0]
    id_traj = idx[1]
    if len(rows) > 2:
        a = rows.copy()
        a.rename({'Latitude': 'source_lat', 'Longitude': 'source_lon', 'Altitude': 'source_alt', 'datetime': 'datetime_start'}, axis=1, inplace=True)
        b = rows.shift(-1)
        b.rename({'Latitude': 'dest_lat', 'Longitude': 'dest_lon', 'Altitude': 'dest_alt', 'datetime': 'datetime_end'}, axis=1, inplace=True)
        pairs = pd.concat([a, b], axis=1)
        pairs.dropna(subset=['dest_lat'], inplace=True) # pour supprimer la dernière paire qui contient forcément un nan
        pairs = pairs.loc[:, ~pairs.columns.duplicated()] # pour supprimer les colonnes duppliquées avec le même nom
        segments.append(pairs)

df_segments = pd.concat(segments)
print(df_segments.shape)
df_segments.head()

**Affichez le nombre de trajectoires par id**

In [ ]:
df_segments.groupby('id')['id_traj'].nunique()
df_segments.groupby('id').size()

b) Créez un geojson à partir du dataframe de segments à l'aide de la fonction `get_traj` en exécutant le code ci-dessous

In [ ]:
def get_traj(df):
    df = df.sort_values('datetime', ascending=True)
    features = []
    for idx, rows in df.groupby(['id', 'id_traj']):
        trajs = []
        for i, row in rows.iterrows():
            trajs.append([
                row['Longitude'],
                row['Latitude'],
                0,
                int(row['datetime'].replace(tzinfo=timezone.utc).timestamp())
            ])
        features.append({
            "type": "Feature",
            "properties": {
                "ID": 'idx_' + str(idx[0]) + '_traj_' + str(idx[1])
            },
            "geometry": {
                "type": "LineString",
                "coordinates": trajs
            }
        })
    trajs_geojson = {
        "type": "FeatureCollection",
        "features": features
    }
    return json.dumps(trajs_geojson)

trajs = get_traj(df)
trajs[:500]

c) Chargez le dataframe de segments (`df_segments`) et le geojson de trajets (`trajs`) en utilisant la fonction [add_data](https://docs.kepler.gl/docs/keplergl-jupyter#2-add-data) de Kepler.

Que constatez-vous ?

In [ ]:
mymap = KeplerGl(height=600, data={'df_segments': df_segments})
mymap.add_data(trajs, name='trajectory')
#======= special correction ========
with open('/workspace/data/ma_config_correction_2.json', "r") as f:
    conf = json.load(f)
mymap.config = conf
#===================================
mymap

d) Affichez le layer de type 'Line' et attribuez une couleur par ID

e) Depuis l'onglet 'Interactions' de Kepler, ajouter dans le tooltip l'information 'source_lat' et 'source_lon'.

Passez ensuite la souris sur une des lignes.

f) Affichez le layer 'Arc' et la vue en 3D et attribuez une couleur par ID

Comparez le résultat avec le layer de type 'Line'.

g) Masquez les layer 'Line' et 'Arc' et affichez le layer 'Trip'

Regardez à la date du 09/02/2009 à 7h à vitesse 0,001

# IV. Clustering
Cette partie permet d'appliquer des techniques de machine learning non supervisé afin de regrouper les lieux d'intérets (cluster) à partir des données géospatiales.

Extraire des lieux d'intéret permet ensuite d'inférer des zones de vies ou des lieux habituels.

In [ ]:
import numpy as np
from math import pi
from functools import partial

import pyproj
from shapely.ops import transform
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
from sklearn.cluster import DBSCAN

a) Convertissez les colonnes 'Latitude' et 'Longitude' en radians à l'aide de la librairie [numpy](https://numpy.org/doc/stable/reference/generated/numpy.radians.html).

Vous nommerez les colonnes résultantes `Latitude_rad` et `Longitude_rad`.

In [ ]:
df['Latitude_rad'] = df['Latitude'].apply(np.radians)
df['Longitude_rad'] = df['Longitude'].apply(np.radians)

b) Les hyperparamètres suivants permettent de préciser une distance en mètre comme paramètre du clustering

In [ ]:
MAX_DIST_M = 20
MIN_SAMPLES = 15

EARTH_RADIUS = 6371088 # mètres

def m2rad(dist):
    return dist / EARTH_RADIUS

dbscan_args = {
    'eps': m2rad(MAX_DIST_M),
    'min_samples': MIN_SAMPLES,
    'metric': 'haversine'
}
dbscan_args

c) Entrainez l'algorithme de clustering dbscan sur les colonnes `Latitude_rad` et `Longitude_rad`.

On pourra passer les arguments `dbscan_args` à la fonction DBSCAN en mettant `**dbscan_args`

In [ ]:
clustering = DBSCAN(**dbscan_args)
clustering.fit(df[['Latitude_rad', 'Longitude_rad']])

d) Ajoutez une colonne 'cluster' contenant les ids des clusters.

Indice : utiliser l'agument 'labels_'

Affichez le nombre de points par cluster.

In [ ]:
df['cluster'] = clustering.labels_

In [ ]:
df['cluster'].value_counts()

e) Visualisez les points associés aux cluster avec Kepler, avec une couleur par 'cluster'.

In [ ]:
KeplerGl(height=600, data={'df_cluster': df})

f) Construction de l'enveloppe des clusters

**Ajoutez une colonne 'geometry' composé d'un polygone de cercle ayant un rayon similaire au paramètre 'eps' du DBSCAN en exécutant le code ci-dessous**

In [ ]:
#========= Version simplifiée, sans projection =========
df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']).buffer(m2rad(MAX_DIST_M) * (180 / pi)), axis=1)
df

In [ ]:
#df['geometry'] = df['geometry'].apply(lambda poly: str(poly) if not isinstance(poly, str) else poly)
#KeplerGl(height=600, data={'df_cluster': df})

In [ ]:
#========= OPTIONNEL : prise en compte de la projection WGS84 =========
#def get_circle_from_point_with_projection(lat, lon, radius):
#    # Shapely point
#    point = Point(lon, lat)
#    # Local azimuthal projection (planar)
#    local_az_proj = f"+proj=aeqd +R=6371000 +units=m +lat_0={point.y} +lon_0={point.x}"
#    # Transfrom WGS84 data to local frame of reference
#    wgs84_to_aeqd = partial(
#        pyproj.transform,
#        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
#        pyproj.Proj(local_az_proj)
#    )
#    # Transfrom local frame of reference data to WGS84
#    aeqd_to_wgs84 = partial(
#        pyproj.transform,
#        pyproj.Proj(local_az_proj),
#        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
#    )
#    # Transform point
#    point_transformed = transform(wgs84_to_aeqd, point)
#    buffer = point_transformed.buffer(radius)
#    buffer_wgs84 = transform(aeqd_to_wgs84, buffer)
#    return buffer_wgs84
#
#df['geometry'] = df.apply(lambda row: get_circle_from_point_with_projection(row['Longitude'], row['Latitude'], m2rad(MAX_DIST_M) * (180 / pi)), axis=1)

**Dans un nouveau DataFrame `clusters_poly`, Fusionnez chaque polygon de cercle d'un cluster en un seul polygone pour créer une enveloppe pour chaque cluster.**

Indice : utilisez 'unary_union' de 'shapely.ops' préalablement importée pour construire un polygone résultant de l'union des polygones de cercles d'un cluster. Il faudra grouper par la colonne `cluster` et appliquer cette fonction à la colonne `geometry`

In [ ]:
clusters_poly = df.groupby('cluster')['geometry'].apply(unary_union).reset_index()
clusters_poly

**Filtrez le DataFrame `cluster_poly` pour éliminer le cluster -1 de l'algorithme dbscan.**

In [ ]:
clusters_poly = clusters_poly[clusters_poly['cluster'] != -1]

**Convertissez la colonne geometry du DataFrame `cluster_poly` en string.**

In [ ]:
clusters_poly['geometry'] = clusters_poly['geometry'].apply(lambda poly: str(poly) if not isinstance(poly, str) else poly)

g) Affichez les polygones correspondants aux enveloppes avec le mode 'polygon', avec une couleur par polygone de cluster.

In [ ]:
KeplerGl(height=600, data={'df_cluster': clusters_poly})

h) A quoi correspondent les clusters ?

### BONUS :
- Faire varier les paramètres 'epsilon' et 'min_sample' de l'algorithme. Que remarquez vous ? Quelles sont d'après vous les limites d'une approche DBSCAN ?
- Pour chaque cluster, appliquer un ratio heure de nuits / heures total via des tranches horaires sur les points présents dans ce cluster. Visualiser ensuite les polygones des clusters avec une couleur variant du rouge au bleu en fonction du ratio. Bleu pour la nuit, rouge pour le jour.